In [ ]:
from os import path
from pprint import pprint
from corputil import FileCorpus
from corputil.utils import load_stopwords
from gensim.models import LdaMulticore
from gensim.models.phrases import Phrases
from gensim.corpora import Dictionary, MmCorpus

stopwords = load_stopwords(path.join('data', 'german.txt'))

In [ ]:
num_topics = 40
chunksize, iterations, passes = 100, 500, 20
labels = ['2015KW44', '2015KW45', '2015KW46', '2015KW47', '2015KW48', '2015KW49', '2015KW50']
files = [path.join('data', 'CurrentNews', 'Corpus_{}.txt').format(label) for label in labels]
output_model = [path.join('models', 'lda', '{}.lda').format(label) for label in labels]
output_dict = [path.join('models', 'lda', '{}.dict').format(label) for label in labels]
output_mm = [path.join('models', 'lda', '{}.mm').format(label) for label in labels]
corpora = [FileCorpus(file) for file in files]

def train_lda(train):
#     corpus = [trigram[bigram[doc]] for doc in train]
    corpus = [doc for doc in train]
    dictionary = Dictionary(corpus)
    dictionary.filter_extremes()
    dictionary.compactify()

    mmCorpus = [dictionary.doc2bow(doc) for doc in corpus]
    lda = LdaMulticore(mmCorpus, id2word=dictionary, chunksize=chunksize, batch=True,
                       num_topics=num_topics, workers=2, passes=passes, iterations=iterations)
    return (dictionary, mmCorpus, lda)

In [ ]:
# bigram = Phrases.load(path.join('models', 'phrase', 'Bigram.phrase'))
# trigram = Phrases.load(path.join('models', 'phrase', 'Trigram.phrase'))

for i, corpus in enumerate(corpora):
    dictionary, mmCorpus, model = train_lda(corpus.doc_token(stopwords=stopwords))
    model.save(output_model[i])
    dictionary.save(output_dict[i])
    MmCorpus.serialize(output_mm[i], mmCorpus)